# Récupération des urls de téléchargement (études) et meta-données associées

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import re
import time
import pandas as pd 

Récupération CSV

In [2]:
cheminLiens = ""

In [3]:
liensEtude = pd.read_csv(cheminLiens+"etude_bretagne.csv", delimiter=';')
idDone = list(liensEtude['html_id'])
#To delette (dossier presenté différemment)
idDone.append('ygtv113')
idDone.append('ygtv129')

In [5]:
#To delette (dossier presenté différemment)
idDone.append('ygtv113')
idDone.append('ygtv129')

In [6]:
liensEtude.head(5)

,id,titre_projet,url_etude,titre_fichier,html_id
0,2017-004867,AU éolien Bois Morand (35),https://travail-collaboratif.din.developpement...,35-P&T-SaintMorand-4-2-EtudeImpacts_COMPL.comp...,ygtv16
1,2017-005464,Ecopôle de Lanvian à Guipavas (29),https://travail-collaboratif.din.developpement...,ECO-PÔLE LANVIAN - DOSSIER DE CREATION (pour a...,ygtv18
2,2017-005501,Plate-forme logistique Tinténiac (35),https://travail-collaboratif.din.developpement...,Etude d'impact_30112017.pdf,ygtv19
3,2017-005685,EARL du Héran à Muel (35),https://travail-collaboratif.din.developpement...,HERAN Muel ETUDE IMPACT IED sept17.pdf,ygtv20
4,2018-004671,Implantation d'un parc éolien - Le Menec à Lou...,https://travail-collaboratif.din.developpement...,22-P&T-Le_Ménec-4-2-EtudeImpacts2.pdf,ygtv21


Connexion à Alfresco avec remplissage formulaire

In [ ]:
identifiant = ""
motdepasse = ""

In [7]:
path = '/Users/js/Documents/stage/scrapping/chromedriver'
driver = webdriver.Chrome(path)
time.sleep(0.5)
driver.get("https://travail-collaboratif.din.developpement-durable.gouv.fr/share/page/site/MRAE-Bretagne/dashboard")
time.sleep(0.5)
pseudo = driver.find_element_by_id("page_x002e_components_x002e_slingshot-login_x0023_default-username")
pseudo.send_keys(identifiant)
time.sleep(0.5)
mdp = driver.find_element_by_id("page_x002e_components_x002e_slingshot-login_x0023_default-password")
mdp.send_keys(motdepasse)
time.sleep(0.5)
submit = driver.find_element_by_id("page_x002e_components_x002e_slingshot-login_x0023_default-submit")
submit.click()

Go to dossier

In [9]:
driver.find_element_by_xpath('//*[@id="HEADER_SITE_DOCUMENTLIBRARY_text"]/a').click()
time.sleep(6)
driver.find_element_by_id("ygtvlabelel5").click()
time.sleep(6)
driver.find_element_by_id("ygtvlabelel10").click()

Récupère la liste des dossiers et leur ids html

In [10]:
elements = driver.find_elements_by_xpath('//*[@id="ygtvc10"]/div')
htmlIDs = [element.get_attribute("id") for element in elements][1:-2]#pour ne pas prendre le premier élément ni le dernier qui ne sont pas des dossiers mais des exemples
htmlIDs = [htmlID for htmlID in htmlIDs if htmlID not in idDone]#filtre les dossiers déjà traité

Fonction utile pour alléger le code

In [11]:
def getHiddenUrl(fichier):
    hiddenElement = fichier.find_element_by_xpath('./td[5]')
    actions = ActionChains(driver)
    actions.move_to_element(hiddenElement)
    actions.perform()
    time.sleep(1)
    return hiddenElement.find_element_by_css_selector("a").get_attribute("href")

In [12]:
def getSize(infos_file):
    tmp = infos_file.find_element_by_xpath('./div/div')
    tmp = re.search('(\d+)\s*(\w+)', tmp.find_elements_by_xpath('./span')[-1].text)
    coef = 1
    if('Mo' in tmp[2] or 'ko' in tmp[2]):
        coef = 1000
    elif('Go' in tmp[2] or 'go' in tmp[2]):
        coef = 1000000
    return int(tmp[1]) * coef

Récupération des études d'impact pour chaque element (avec l'avis si disponible)

In [15]:
for htmlID in htmlIDs:
    print(htmlID)
    
    
    time.sleep(3)
    element = driver.find_element_by_id(htmlID)
    tmp = re.search('(\d+-\d+)[\s_-]*((?:.|\s)+)', element.text)
    idDoc = tmp[1]
    titre_projet = tmp[2]
    element.click()#click sur le dossier
    time.sleep(6)
    driver.find_element_by_id(htmlID[:4]+"c"+htmlID[4:]).find_element_by_css_selector('div').click()#click sur le sous dossier "dossier"
    time.sleep(6)
    table = driver.find_elements_by_css_selector("table[summary]")[-1] #récupération de la table contenant les fichiers
    fichiers = table.find_elements_by_css_selector("tbody")[1].find_elements_by_css_selector("tr")#récupération des fichiers du dossier
    
    max_size = -1
    url_max = ""
    findEtude = False
    for fichier in fichiers:
        
        infos_file = fichier.find_element_by_xpath('./td[4]')#Récupération de la "<td>" contenant les infos
        
        #extraction du nom de fichier et de l'url de téléchargement
        name_file = infos_file.find_element_by_css_selector('h3').text
        name_file_norm = name_file.lower().replace("é","e")
        url_file = getHiddenUrl(fichier)
        
        #Extraction taille fichier
        size_file = getSize(infos_file)
        
        #Véréficiation max
        if(max_size<size_file):
            max_size = size_file
            url_max = url_file
        
        #Si fichier est identifié comme étude d'impact
        if("etude" in name_file_norm and 'impact' in name_file_norm and "rnt" not in name_file_norm):
            findEtude = True
            dataCSV = idDoc+";"+titre_projet+";"+url_file+";"+name_file+";"+str(htmlID)+"\n"
            file = open(cheminLiens+'etude_bretagne.csv','a')
            file.write(dataCSV)
            file.close()
            
    
    #Si on n'a pas identifié le fichier de l'étude grace au nom, on prend le fichier le plus volumineux et on indique que le fichier est à vérifier        
    if(findEtude==False):
        dataCSV = idDoc+";"+titre_projet+";"+url_max+";"+name_file+"_not_shure;"+str(htmlID)+"\n"
        file = open(cheminLiens+'etude_bretagne.csv','a')
        file.write(dataCSV)
        file.close()
        
driver.close()